In [1]:
# Imports
import random
import string
import tempfile
import shutil
import subprocess as sp
import os

from PIL import Image
from torch import nn as nn
from torch import Generator
from torch.utils.data import random_split,DataLoader
# TODO: Currently assumes that pytest runs from project root
from util.car_dataset import check_dir, check_file, CarDataset, DatasetError
from networks.common import *
from networks.CNNbase import CNNBasic

In [2]:
# Helper Functions
def rand_string(length=5):
    return ''.join(random.choices(string.ascii_letters +
        string.digits, k=length))

def loaded_dataset(temp_dir):
    '''
    Loads the car_dataset and returns the CarDataset object.
    '''
    #image_folder_list = ['0_Frames','1_Frames','2_Frames','3_Frames']
    #label_file_list = ['0.txt','1.txt','2.txt','3.txt']
    image_folder_list = ['0_Frames']
    label_file_list = ['0.txt']
    car_data = CarDataset(temp_dir,image_folder_list,label_file_list)
    car_data.drop_nan()

    return car_data

def dataset_loader(loaded_dataset):
    length = len(loaded_dataset)
    split_set = random_split(loaded_dataset,[round(0.80*length)
                ,round(0.20*length)],generator=Generator().manual_seed(42))
    train_loader = DataLoader(split_set[0],batch_size=20)
    test_loader = DataLoader(split_set[1],batch_size=20)
    '''
    Lowering values to sane levels to help run tests on github
    runners. Ideally on local GPU (4GB) 80/20 split with batch size of
    30 works well.
    '''
    return (train_loader,test_loader)

def network():
    network = CNNBasic()
    network = network.to(get_device())
    network.double()
    return network

In [3]:
root = os.getcwd()
print(os.getcwd())

C:\Users\aksha\Desktop\dl-based-sensor-calib


In [4]:
# Load Data
data = loaded_dataset(os.path.join(os.getcwd(),"dummy"))

In [5]:
# Split Dataset
datasets = dataset_loader(data)

In [6]:
# Initialize Network
net = network()

In [7]:
# Train
train = datasets[0]
print("Start Training")
run_training(net,train,1,get_device())

0it [00:00, ?it/s]

Start Training


48it [01:14,  1.56s/it]

[0.0023945062209875597, 0.0021735814722538913, 0.0020342568510803363, 0.001790008499580588, 0.001741319400196628, 0.001776566849462381, 0.0018356273661992606, 0.001780277401897139, 0.0015712888014433994, 0.0014287431208034485, 0.0017053991203078431, 0.001633998732580986, 0.001277412105383961, 0.0014458903253205682, 0.0014783725855323545, 0.0009762948334857834, 0.001117353595888093, 0.0009775080117433415, 0.0011770719658377867, 0.0009869856737355894, 0.000899311560816988, 0.0007822256705433393, 0.00090340678836575, 0.0007801253008806608, 0.00096263780086569, 0.0007653563790641925, 0.0007641845554145912, 0.0007129695156406157, 0.0008104611854390814, 0.0006860670817180272, 0.0006220820009371797, 0.0007759756080550086, 0.0005339381539605198, 0.0005557450860693784, 0.0007455381600232448, 0.0007687409320583085, 0.0004971577505200862, 0.0006251108197948841, 0.0003951384427874184, 0.00044137909824217426, 0.0005875411301488106, 0.0005322422458625468, 0.0005609713665284566, 0.0004856531255728606

In [8]:
# Test
test = datasets[1]
print("Start Inference")
run_inference(net,test,get_device())

Start Inference
Test Set Average Loss 0.0017
